# 0.标准输入输出

## 输入
1.n,k=map(int,input().strip().strip())

## 1.哈希表


### 1.转换思路之 数组中选两数求和为目标值---a+b=target
- 创建一个字典，将nums中的元素作为key，索引作为value
- 遍历nums，判断target-nums[i]是否在字典中，是则返回结果,否，则将nums[i]作为key，索引作为value加入字典!
- 时间复杂度：O(n)
- 空间复杂度：O(n)

In [ ]:
n,target=map(int,input().split())
nums=list(map(int,input().split()))
dic={}
for i in range(n):
    complement=target-nums[i]
    if complement in dic:
        print(dic[complement],i)
        exit()
    dic[nums[i]]=i

### 2.怎么选取健和值--字母异位词分组
思路:
1. 创建一个字典，key为排序后的字符串，value为列表，列表中存放的是原字符串---这样异位词组的健就是一样的!
=================================================================================
2. 遍历字符串列表，将字符串排序，并作为key，将原字符串作为value添加到字典中
3. 遍历字典，将value列表中的字符串添加到一个新的列表中，并返回新的列表

学:
1.注意!:sorted(s) 返回的是列表（list），而列表是不可哈希的,所以不能作为字典的key--要用key="".join(sorted(s))
2.用collections中的defaultdict来创建字典--默认字典，字典的key不存在时，会创建一个空的列表，并返回该列表
3."".join(Listt)来将列表Listt中的元素连接成一个字符串

In [ ]:
from collections import defaultdict
class solution:
    def groupanagrams(self,strs):
        mp=defaultdict(list)
        for s in strs:
            key="".join(sorted(s))
            mp[key].append(s)
        return list(mp.values())
if __name__=="__main__":
    n=int(input())
    strs=input().split()
    so=solution()
    res=so.groupanagrams(strs)
    for group in res:
        #print(" ".join(group))
        print(*group)

### 3.巨帅的贪心哈希解法--最长连续序列

学:

1.**利用set(nums)方便查找nums[i]-1/nums[i]+1是否在nums中---set()查找效率高O(1)!!!!!**

2.nums[i]-1不在set(nums)中说明其就是一个序列的起点

3.nums[i]+1在set(nums)中,说明该序列长度可以增加--le+=1,一直到找不到为止就是nums[i]序列的最终连续长度

4.maxlen = max(maxlen, le)找到最长长度

In [ ]:
from collections import deque
class solution:
    def maxsequence(self,nums):
        maxle=0
        setnums=set(nums)
        for i in range(len(nums)):
            if (nums[i]-1) not in setnums:
                le=1
                while (nums[i]+le) in setnums:
                    le+=1
                maxle=max(maxle,le)
            else:
                continue
        return maxle

so=solution()
n=int(input())
nums=list(map(int,input().split()))
print(so.maxsequence(nums))


### 4.技巧--字符为key,出现次数为value
dict.get(key, default) 可以获取键 key 对应的值，若键不存在则返回 default（这里用 0 作为默认值）。


In [1]:
from collections import defaultdict

class Solution:
    def allsubword(self, s, p):
        ns, np = len(s), len(p)
        dic = {}  # 存储p中字符的出现次数
        for char in p:  # 直接遍历p中的每个字符（比用索引更直观）
            # 若char在dic中，获取其当前值+1；若不在，用0+1（即初始化为1）
            dic[char] = dic.get(char, 0) + 1
        return dic  # 测试用，实际可根据需求处理

5.技巧--删除字典中的健

In [2]:
del scount[s[i-np]]

NameError: name 'scount' is not defined

### 6.技巧使用Counter
from collections import Counter

Counter 的内容以 键值对 的形式存储，与普通字典类似：
键（key）：被计数的元素（比如字符串中的每个字符、列表中的每个元素等）。
值（value）：该元素出现的次数（非负整数）。

## 2.双指针

### 1.同起点/紧挨着的双指针--移动零

两种思路:
1.挨个判断0移动到后面---需要考虑更多边界条件
2.移动非0到前面---更简洁,但会自己移动自己

In [ ]:
# 移动0 -后移
class solution:
    def movezero(self,nums):
        left=0
        right=1
        while right<len(nums):
            if nums[left]==0 and nums[right]!=0:
                nums[left],nums[right]=nums[right],nums[left]
                right+=1
                left+=1
            elif nums[left]==0:
                if nums[right]==0:
                    right+=1
            else:
                right+=1
                left+=1
so=solution()
n=int(input())
nums=list(map(int,input().split()))
so.movezero(nums)
print(*nums)



In [ ]:
# 移动非0--前移
class Solution:
    def moveZeroes(self, nums):
        n = len(nums)
        i = 0  # 记录非零元素应放置的位置
        
        # 遍历数组，将非零元素移到前面
        for j in range(n):
            if nums[j] != 0:
                nums[i], nums[j] = nums[j], nums[i]  # 交换非零元素到前面
                i += 1  # 移动非零元素位置索引

# 读入数据并调用函数
n = int(input().strip())
nums = list(map(int, input().split()))
Solution().moveZeroes(nums)
print(" ".join(map(str, nums)))


### 2.两头指针移动之优化(贪心)多移动其中一头--盛水最多容器

学:
1.我在想优化双指针，移动指针的时候跳过比当前短板更短的时候,循环没写好容易写成死循环,要记住这样写!:
    cur_left=nums[left] #要在循环外定义cur_left,方便比较
    while left < right and nums[left]<=cur_left:
                    left+=1 #只在循环里面叠加
        不要忘记判断left<right且将nums[left]<=cur_left:放入同级别的while中

In [ ]:
class solution:
    def maxwater(self,nums):
        left=0
        right=len(nums)-1
        res=0        
        while left<right:
            v=min(nums[left],nums[right])*(right-left)
            res=max(res,v)
            if nums[left]<nums[right]:
                cur_left=nums[left]
                while left < right and nums[left]<=cur_left:
                    left+=1
                    
            else:
                cur_right=nums[right]
                while left < right and nums[right]<=cur_right:
                    right-=1
                        
        return res
so=solution()
n=int(input())
nums=list(map(int,input().split()))
print(so.maxwater(nums))
                

### 3.三个指针（去重写法）--三数之和

学：

1.固定一个指针i，滑动窗口两个指针k=i+1,j=len(nums)-1开始滑

2.每次i的循环注意k=i+1,j=len(nums)-1要重新初始化

3.去重的时候主要在（1）i的外循环注意and边界条件i<len(nums）-2)（2）内循环total=0的时候（注意and边界条件k<j）


In [ ]:
class Solution:
    def threeSum(self, nums):
        """
        三数之和，双指针法
        :param nums: List[int] 代表输入数组
        :return: List[List[int]] 符合条件的三元组列表
        """
        nums.sort()  # 先排序
        n = len(nums)
        res = []

        for i in range(n - 2):
            if i > 0 and nums[i] == nums[i - 1]:  # 跳过重复元素
                continue

            left, right = i + 1, n - 1  # 左右指针
            while left < right:
                total = nums[i] + nums[left] + nums[right]
                if total < 0:
                    left += 1
                elif total > 0:
                    right -= 1
                else:
                    res.append([nums[i], nums[left], nums[right]])
                    # 跳过重复元素
                    while left < right and nums[left] == nums[left + 1]:
                        left += 1
                    while left < right and nums[right] == nums[right - 1]:
                        right -= 1
                    left += 1
                    right -= 1
        return res

# 读取输入
if __name__ == "__main__":
    n = int(input())  # 读取 n
    nums = list(map(int, input().split()))  # 读取数组
    solution = Solution()
    results = solution.threeSum(nums)
    for triplet in results:
        print(" ".join(map(str, triplet)))


### 1.在滑动（滑动得前缀后缀max）的时候就根据贪心计算当前值--接雨水

学：

1.可以边滑动边计算值的原因：
当 height[left] < height[right] 时，我们可以确定：
右侧一定存在一个比 height[left] 更高的柱子（至少 height[right] 就是）。
因此，left 位置的存水量只取决于左侧的最大高度（left_max），因为右侧的最大高度无论如何都不会小于 height[right]（而 height[right] 已经比 height[left] 大了）

省空间！
利用了贪心思想

In [ ]:
class Solution:
    def trap(self, height):
        if not height:
            return 0
        
        left, right = 0, len(height) - 1
        left_max, right_max = 0, 0
        water = 0
        
        while left < right:
            if height[left] < height[right]:
                if height[left] >= left_max:
                    left_max = height[left]
                else:
                    water += left_max - height[left]
                left += 1
            else:
                if height[right] >= right_max:
                    right_max = height[right]
                else:
                    water += right_max - height[right]
                right -= 1
        
        return water

# 读取输入并处理
if __name__ == "__main__":
    n = int(input())
    height = list(map(int, input().split()))
    solution = Solution()
    print(solution.trap(height))


## 3.滑动窗口（双指针的子集）-双指针+连续区间

### 1.滑动窗口的left可以用for循环，right用while循环（逻辑清楚）--求无重复的最长子串

学：

1.用集合set()来判断重复元素--查找，添加，删除的时间复杂度都是O(1)

2.s=str(input())或者s= input()--字符串本身就是可迭代的，不需要用list()



### 2.单调队列/二叉堆辅助--滑动窗口的最大值

双端单调队列的求法:


In [ ]:
from collections import deque
import sys

class Solution:
    def maxSlidingWindow(self, nums, k):
        deque_window = deque()  # 双端队列
        result = []
        
        for i in range(len(nums)):
            # 移除窗口左侧已过期的元素索引
            if deque_window and deque_window[0] < i - k + 1:
                deque_window.popleft()
            
            # 保持队列单调递减，移除所有小于当前元素的索引
            while deque_window and nums[deque_window[-1]] < nums[i]:
                deque_window.pop()
            
            # 添加当前元素索引
            deque_window.append(i)
            
            # 记录最大值（窗口形成后）
            if i >= k - 1:
                result.append(nums[deque_window[0]])

        return result

if __name__ == "__main__":
    # 读取输入
    n, k = map(int, sys.stdin.readline().split())
    nums = list(map(int, sys.stdin.readline().split()))
    
    # 计算滑动窗口最大值
    solution = Solution()
    result = solution.maxSlidingWindow(nums, k)
    
    # 输出结果
    print(" ".join(map(str, result)))


二叉堆解法

In [ ]:
from collections import deque
import sys

class Solution:
    def maxSlidingWindow(self, nums, k):
        deque_window = deque()  # 双端队列
        result = []
        
        for i in range(len(nums)):
            # 移除窗口左侧已过期的元素索引
            if deque_window and deque_window[0] < i - k + 1:
                deque_window.popleft()
            
            # 保持队列单调递减，移除所有小于当前元素的索引
            while deque_window and nums[deque_window[-1]] < nums[i]:
                deque_window.pop()
            
            # 添加当前元素索引
            deque_window.append(i)
            
            # 记录最大值（窗口形成后）
            if i >= k - 1:
                result.append(nums[deque_window[0]])

        return result

if __name__ == "__main__":
    # 读取输入
    n, k = map(int, sys.stdin.readline().split())
    nums = list(map(int, sys.stdin.readline().split()))
    
    # 计算滑动窗口最大值
    solution = Solution()
    result = solution.maxSlidingWindow(nums, k)
    
    # 输出结果
    print(" ".join(map(str, result)))


## 4.子串技巧汇总

### 1.前缀和技巧（结合哈希表应对前缀和出现重复的情况--sums中有负数）--和为k的子数组(连续)
哈希表记录健为presum[i]出现的次数!
dic[presum[i]]=dic.get(presum[i],0)+1  


学规范

前缀和代码标准：
        n=len(nums)
        presum=[0]*(n+1)
        for i in range(1,n+1):
            presum[i]=presum[i-1]+nums[i-1]

In [ ]:
class Solution:
    def sumksubseq(self,nums,k):
        n=len(nums)
        presum=[0]*(n+1)
        dic={0:1}
        res=0
        for i in range(1,n+1):
            presum[i]=presum[i-1]+nums[i-1]
            res+=dic.get(presum[i]-k,0)
            dic[presum[i]]=dic.get(presum[i],0)+1            
        return res

if __name__ == "__main__":
    n,k=map(int,input().strip().split())   
    nums = list(map(int, input().strip().split()))
    solution = Solution()
    print(solution.sumksubseq(nums, k))
